In [1]:
import requests
import pandas as pd
import time
import random
from github import Github, GithubException
from datetime import datetime
import psycopg2
import os

In [4]:
#leer el csv en la carpeta actual
#Agregar un archivo que está en github
#df_busqueda = pd.read_csv('https://github.com/labdatos-se/economia/blob/main/mapa_gro/gro_bodega_chedraui.csv', encoding='latin-1')
df_busqueda = pd.read_csv('D:\Proy\GitHub\labdatos-se\economia\mapa_gro\gro_bodega_chedraui.csv', encoding='latin-1')
#df_busqueda = pd.read_csv('guerrero.csv')
df_busqueda.head()

,producto,marca,precio,tienda,fecha_consulta,busqueda
0,aceite 123 vegetal 123 1l,123,47.0,chedraui acapulco costera,37:46.6,aceite
1,aceite patrona mixto 1 l,patrona,45.0,chedraui acapulco costera,37:46.6,aceite
2,aceite oliva selecto brand puro de orujo pet 1l,selecto brand,132.0,chedraui acapulco costera,37:46.6,aceite
3,aceite en aerosol pam antiadherente original ...,NaN,69.0,Bodega Aurrera Chilpancingo Norte,38:19.9,aceite
4,aceite vegetal 123 1 l,NaN,45.0,Bodega Aurrera Chilpancingo Norte,38:19.9,aceite


In [18]:
#Función de limpieza de prueba
def limpiaprod(df):
    #Aceite
    aceite = df[df['busqueda'].str.contains("aceite") & (df['producto'].str.contains("aceite"))]
    aceite = aceite[~aceite["producto"].str.contains("oliva")]

    #Arroz
    arroz = df[df['busqueda'].str.contains("arroz") & (df['producto'].str.contains("arroz"))]
    arroz = arroz[~arroz["producto"].str.contains("sushi") & ~arroz["producto"].str.contains("arborio") & ~arroz["producto"].str.contains("precocido")& ~arroz["producto"].str.contains("orgánico")]

    #Atún
    atún = df[df['busqueda'].str.contains("atún") & (df['producto'].str.contains("agua")) & (df['producto'].str.contains("140"))]
    atún = atún[~atún["producto"].str.contains("aceite")]

    #Azúcar
    azúcar = df[df['busqueda'].str.contains("azúcar") & df['producto'].str.contains("azúcar") & (df['producto'].str.contains("2"))]

    #Cepillo tipo plancha
    cepillo_plancha = df[df['producto'].str.contains("cepillo tipo plancha")]
    cepillo_plancha = cepillo_plancha[~cepillo_plancha["producto"].str.contains("cja")]

    #cepillo dental
    cepillo_dental = df[df['producto'].str.contains("cepillo dental") & (df['busqueda'].str.contains("cepillo"))]
    cepillo_dental = cepillo_dental[~cepillo_dental["producto"].str.contains("infantil") & ~cepillo_dental["producto"].str.contains("eléctrico") & ~cepillo_dental["producto"].str.contains("kid") & ~cepillo_dental["producto"].str.contains("pasta") & ~cepillo_dental["producto"].str.contains("electrónico") & ~cepillo_dental["producto"].str.contains("pow")]

    #chocolate en polvo
    #REVISAR
    chocolate = df[(df['producto'].str.contains("chocolate en polvo") | df['producto'].str.contains("polvo sabor chocolate")) & (df['producto'].str.contains("700 g"))]
    chocolate = chocolate[~chocolate["producto"].str.contains("cereal")]   
    
    #cloro
    cloro = df[df['producto'].str.contains("cloro") & (df['busqueda'].str.contains("cloro"))]
    cloro = cloro[~cloro["producto"].str.contains("toallitas") & ~cloro["producto"].str.contains("tricloro") & ~cloro["producto"].str.contains("sin") & ~cloro["producto"].str.contains("detergente") & ~cloro["producto"].str.contains("polvo") & ~cloro["producto"].str.contains("alternativa") & ~cloro["producto"].str.contains("anti") & ~cloro["producto"].str.contains("con") & ~cloro["producto"].str.contains("perla") & ~cloro["producto"].str.contains("suplemento") & ~cloro["producto"].str.contains("tableta") & ~cloro["producto"].str.contains("piramina") & ~cloro["producto"].str.contains("crema")]      
    
    #desodorante
    desodorante = df[(df['producto'].str.contains("desodorante") | df['producto'].str.contains("antitranspirante")) & df['producto'].str.match("barra 50 g")]

    #escobas
    escobas = df[df['producto'].str.contains("escoba") & (df['producto'].str.match("unidad") | df['producto'].str.match("pieza") | df['producto'].str.match("1 pieza")) & (df['busqueda'].str.contains("escobas"))]
    escobas = escobas[~escobas["busqueda"].str.contains("fibra")]

    #fibra
    fibra = df[df['producto'].str.contains("fibra") & df['busqueda'].str.match("fibra") & (df['producto'].str.match("unidad") | df['producto'].str.match("1 pieza"))]
    fibra = fibra[~fibra["producto"].str.contains("paño")]

    #franelas
    franelas = df[df['producto'].str.contains("franela")]

    #café
    café = df[df['busqueda'].str.contains("café") & (df['producto'].str.contains("frasco 225 g") | df['producto'].str.contains("frasco 200 g") | df['producto'].str.contains("frasco 190g ") | df['producto'].str.contains("frasco 180 g") | df['producto'].str.contains("frasco 175 g"))]
    café = café[~café["producto"].str.contains("con") & ~café["producto"].str.contains("pack")]

    #frijol
    frijol = df[(df['busqueda'].str.contains("frijol")) & (df['producto'].str.contains("frijol")) & (df['producto'].str.contains("1 kg") | df['producto'].str.contains("900 g"))]
    frijol = frijol[~frijol["producto"].str.contains("refritos")]

    #guantes de plástico
    guantes = df[df['producto'].str.contains("guantes") & (df['busqueda'].str.contains("guantes"))]

    #harina para atole
    harina = df[df['producto'].str.contains("harina de arroz") & df['busqueda'].str.match("harina") & (df['producto'].str.contains("200 g"))]

    #jabón de barra
    jabón_barra = df[(df['producto'].str.contains("jabón")) & (df['producto'].str.contains("150 g") & (df['producto'].str.contains("barra")))]
    jabón_barra = jabón_barra[~jabón_barra["producto"].str.contains("derm") & ~jabón_barra["producto"].str.contains("facial")]

    #jabón en polvo
    jabón_polvo = df[(df['producto'].str.contains("detergente")) & (df['producto'].str.contains("1 kg") | df['producto'].str.contains("2 kg") | df['producto'].str.contains("2.1 kg") | df['producto'].str.contains("750 g") | df['producto'].str.contains("800 g"))]

    #jalador
    jalador = df[df['producto'].str.contains("jalador") & (df['busqueda'].str.contains("jalador")) & (df['producto'].str.contains("1"))]
    jalador = jalador[~jalador["producto"].str.contains("vidrio") & ~jalador["producto"].str.contains("inye") & ~jalador["producto"].str.contains("paño")]

    #jergas
    jergas = df[df['producto'].str.contains("jerga") & (df['busqueda'].str.contains("jerga")) & (df['producto'].str.contains("1"))]

    #lata de chiles
    lata_chiles = df[df['producto'].str.contains("chiles") & (df['busqueda'].str.contains("chiles")) & (df['busqueda'].str.contains("lata")) & (df['producto'].str.contains("220 g"))]
    lata_chiles = lata_chiles[~lata_chiles["producto"].str.contains("adob")]

    #lenteja
    lenteja = df[df['producto'].str.contains("lenteja")]# & (df['producto'].str.contains("900 g"))]
    lenteja = lenteja[~lenteja["producto"].str.contains("sopa") & ~lenteja["producto"].str.contains("ensalada") & ~lenteja["producto"].str.contains("chorizo") & ~lenteja["producto"].str.contains("orgánica") & ~lenteja["producto"].str.contains("hamburguesa")]

    #limpiador para pisos
    limpiador_pisos = df[df['producto'].str.contains("limpiador") & (df['busqueda'].str.contains("limpiador"))]
    limpiador_pisos = limpiador_pisos[~limpiador_pisos['busqueda'].str.contains("pieza") & ~limpiador_pisos['busqueda'].str.contains("aerosol") & ~limpiador_pisos['busqueda'].str.contains("atomizador") & ~limpiador_pisos['busqueda'].str.contains("doypack") & ~limpiador_pisos['busqueda'].str.contains("poett") & ~limpiador_pisos['busqueda'].str.contains("pouch") & ~limpiador_pisos['busqueda'].str.contains("spray")]

    #mayonesa
    mayonesa = df[(df['busqueda'].str.contains("mayonesa")) & (df['producto'].str.contains("mayonesa")) & (df['producto'].str.contains("frasco")) & (df['producto'].str.contains("650") | df['producto'].str.contains("725") | df['producto'].str.contains("730") | df['producto'].str.contains("750") | df['producto'].str.contains("775") | df['producto'].str.contains("790") | df['producto'].str.contains("800"))]

    #mermelada
    mermelada = df[df['producto'].str.contains("mermelada") & (df['busqueda'].str.contains("mermelada")) & (df['producto'].str.contains("440"))]

    #pasta dental
    pasta_dental = df[(df['busqueda'].str.contains("pasta")) & (df['producto'].str.contains("pasta")) & (df['producto'].str.contains("125 ml"))]
    pasta_dental = pasta_dental[~pasta_dental["producto"].str.contains("cepillo")]

    #peine
    peine = df[df['producto'].str.contains("peine") & (df['busqueda'].str.contains("peine")) & (df['producto'].str.contains("1"))]
    peine = peine[~peine["producto"].str.contains("drer") & ~peine["producto"].str.contains("mickey") & ~peine["producto"].str.contains("winnie") & ~peine["producto"].str.contains("térmico")]

    #rastrillos
    #rastrillos = df[df['producto'].str.contains("rastrillo") & (df['busqueda'].str.contains("rastrillo")) & (df['producto'].str.contains("3 piezas"))]
    rastrillos = df[df['producto'].str.contains("rastrillo") & (df['busqueda'].str.contains("rastillo")) & (df['producto'].str.contains("3 piezas"))]  #RASTILLO
    rastrillos = rastrillos[~rastrillos["producto"].str.contains("máquina") & ~rastrillos["producto"].str.contains("cartucho")]

    #recogedor
    recogedor = df[df['producto'].str.contains("recogedor") & (df['busqueda'].str.contains("recogedor")) & (df['producto'].str.contains("1"))]

    #rollos de papel
    rollos_papel = df[df['producto'].str.contains("papel") & (df['busqueda'].str.contains("rollos")) & (df['producto'].str.contains("4"))]
    rollos_papel = rollos_papel[~rollos_papel["producto"].str.contains("24") & ~rollos_papel["producto"].str.contains("42")]

    #sacate
    sacate = df[df['busqueda'].str.contains("sacate") & ((df['producto'].str.contains("esponja")) | (df['producto'].str.contains("toalla"))) & (df['producto'].str.contains("1"))]

    #sal
    sal = df[df['busqueda'].str.contains("sal") & (df['producto'].str.contains("sal"))]
    sal = sal[~sal["producto"].str.contains("mantequilla") & ~sal["producto"].str.contains("margarina") & ~sal["producto"].str.contains("parrillera") & ~sal["producto"].str.contains("himalaya") & ~sal["producto"].str.contains("con") & ~sal["producto"].str.contains("sin") & ~sal["producto"].str.contains("botana") & ~sal["producto"].str.contains("fritura") & ~sal["producto"].str.contains("papas") & ~sal["producto"].str.contains("palomitas") & ~sal["producto"].str.contains("cacah") & ~sal["producto"].str.contains("jabón") & ~sal["producto"].str.contains("flor") & ~sal["producto"].str.contains("agua") & ~sal["producto"].str.contains("hornea") & ~sal["producto"].str.contains("cristal")  & ~sal["producto"].str.contains("uva") & ~sal["producto"].str.contains("sustituto") & ~sal["producto"].str.contains("gou") & ~sal["producto"].str.contains("ahumada") & ~sal["producto"].str.contains("alpo") & ~sal["producto"].str.contains("light")]

    #sardina
    sardina = df[df['busqueda'].str.contains("sardina") & (df['producto'].str.contains("sardina")) & (df['producto'].str.contains("425"))]

    #sopas instantáneas
    sopas = df[df['busqueda'].str.contains("sopa") & (df['producto'].str.contains("insta"))]

    #toallas faciales
    toallas_fac = df[df['busqueda'].str.contains("toalla") & (df['producto'].str.contains("facial"))]

    #toallas femeninas
    toallas_fem = df[df['busqueda'].str.contains("toalla") & (df['producto'].str.contains("fem")) & (df['producto'].str.contains("8"))]
    toallas_fem = toallas_fem[~toallas_fem["producto"].str.contains("28") & ~toallas_fem["producto"].str.contains("18")]

    #toallas húmedas
    toallas_hum = df[df['busqueda'].str.contains("toalla") & (df['producto'].str.contains("eda")) & (df['producto'].str.contains("80"))]
    toallas_hum = toallas_hum[~toallas_hum["producto"].str.contains("3")]

    #Concatenar
    #finaldf = pd.concat([aceite, arroz, atún, azúcar, cepillo_plancha, cepillo_dental, chocolate, cloro, desodorante, escobas, fibra, franelas, café, frijol, guantes, harina, jabón_barra, jabón_polvo, jalador, jergas, lata_chiles, lenteja, limpiador_pisos, mayonesa, mermelada, pasta_dental, peine, rastrillos, recogedor, rollos_papel, sacate, sal, sardina, sopas, toallas_fac, toallas_fem, toallas_hum])
    finaldf = pd.concat([azúcar])
    #reset index
    finaldf.reset_index(drop=True, inplace=True)
    return finaldf


In [19]:
limpiaprod(df_busqueda)

,producto,marca,precio,tienda,fecha_consulta,busqueda
0,azúcar zulka morena 2kg,zulka,90.0,chedraui acapulco costera,45:56.1,azúcar
1,azúcar zulka morena 2kg,zulka,93.0,super chedraui teloloapan,47:18.2,azúcar


In [ ]:
#Guardar el dataframe en un csv
limpiaprod(df_busqueda).to_csv('D:\Proy\GitHub\labdatos-se\economia\mapa_gro\corner_limpio.csv', encoding='latin1', index=False)